In [18]:
using MPSGE_MP, JuMP

In [19]:
M = MPSGEModel()

@parameter(M,tax, 0)

@sectors(M,begin
    X
    Y
    W
end)

@commodities(M,begin
    PX
    PY
    PL
    PK
    PW
end)

@consumer(M, CONS)

add_production!(M,
    X,
    ScalarNest(:t; elasticity = 0, children = 
        [ScalarOutput(PX, 120.)]
    ),
    ScalarNest(:s; elasticity = .5, children = [
        ScalarInput(PY,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 40; taxes = [Tax(CONS, tax)]),
            ScalarInput(PK, 60; taxes = [Tax(CONS, tax)])
            ])
        ]
        ) 
)

add_production!(M,
    Y,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PY, 120),
    ]),
    ScalarNest(:s; elasticity = .75, children = [
        ScalarInput(PX,20),
        ScalarNest(:va; elasticity = 1, children = [
            ScalarInput(PL, 60),
            ScalarInput(PK, 40)
        ])
    ])
)


add_production!(M,
    W,
    ScalarNest(:t; elasticity = 0, children = [
        ScalarOutput(PW, 200)
    ]),
    ScalarNest(:s, elasticity = 1, children = [
        ScalarInput(PX, 100),
        ScalarInput(PY, 100),
    ])
)

add_demand!(M,
    CONS,
    [ScalarDem(PW, 200)],
    [
        ScalarEndowment(PL,100),
        ScalarEndowment(PK,100)
    ]
)

jm = build!(M);


# Benchmark

In [20]:
solve!(M; :cumulative_iteration_limit => 0)
df_benchmark = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_5542.tmp
 > cumulative_iteration_limit 0
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris
Preprocessed size   : 9

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     1     1 0.0000e+00           I 0.0e+00 0.0e+00 (f[    2])

Major Iterations. . . . 0
Minor Iterations. . . . 0
Restarts. . . . . . . . 0
Crash Iterations. . . . 0
Gradient Steps. . . . . 0
Function Evaluations. . 1
Gradient Evaluations. . 1
Basis Time. . . . . . . 0.000000
Total Time. . . . . . . 0.000000
Residual. . . . . . . . 0.000000e+00
Postsolved residual: 0.0000e+00


Row,var,value,margin
,GenericV…,Float64,Float64
1,X,1.0,0.0
2,Y,1.0,0.0
3,W,1.0,0.0
4,PY,1.0,0.0
5,PW,1.0,0.0
6,PL,1.0,0.0
7,PX,1.0,0.0
8,PK,1.0,0.0
9,CONS,200.0,0.0


# Counterfactual tax = .5

In [23]:
fix(PW,1)

MPSGE_MP.set_value!(tax,.5)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_8241.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             6.9173e+01             0.0e+00 (f[    7])
    1     1     0     8 2.9618e+00  1.0e+00    0.0e+00 (f[    6])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 2.9618e+00           I 0.0e+00 1.9e+00 (f[    6])
    1     1     3     3 2.4397e-02  1.0e+00 SO 0.0e+00 1.7e-02 (f[   10])
    2     1     4     4 2.5675e-06  1.0e+00 SO 0.0e+00 2.1e-06 (f[   10])
    3     1     5     5 1.2750e-13  1.0e+00 SO 0.0e+00 7.8e-14 (f[    7])

Major Iterations. . . . 3
Minor Iterations. . . . 3
Restarts. . . . . . . . 0
Crash Iteratio

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.861867,-7.81597e-14
2,Y,1.10915,-7.10543e-15
3,W,0.983827,-4.26326e-14
4,PY,0.868686,2.84217e-14
5,PW,1.0,2.84217e-14
6,PL,0.841191,2.84217e-14
7,PX,1.15116,-2.84217e-14
8,PK,0.787062,8.52651e-14
9,CONS,196.765,2.84217e-14


# Counterfactual tax = .1

In [24]:
fix(PW,1)

MPSGE_MP.set_value!(tax,.1)

solve!(M; :cumulative_iteration_limit => 10_000)
df = generate_report(M)

Reading options file C:\Users\MPHILL~1\AppData\Local\Temp\jl_ABC3.tmp
 > cumulative_iteration_limit 10000
Read of options file complete.

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Preprocessed size   : 8

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.4094e+01             0.0e+00 (f[    7])
    1     1     0     8 1.5012e-01  1.0e+00    0.0e+00 (f[    6])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     2     2 1.5012e-01           I 0.0e+00 1.1e-01 (f[    6])
    1     1     3     3 6.2994e-05  1.0e+00 SO 0.0e+00 4.2e-05 (f[   10])
    2     1     4     4 1.7054e-11  1.0e+00 SO 0.0e+00 1.4e-11 (f[   10])

Major Iterations. . . . 2
Minor Iterations. . . . 2
Restarts. . . . . . . . 0
Crash Iterations. . . . 1
Gradient Steps. . . . . 0
Function Evaluations. . 4
Gradient E

Row,var,value,margin
,GenericV…,Float64,Float64
1,X,0.968548,7.46425e-12
2,Y,1.02732,6.86384e-12
3,W,0.999102,2.84217e-13
4,PY,0.967574,-8.2423e-13
5,PW,1.0,-2.78817e-11
6,PL,0.960787,1.98952e-12
7,PX,1.03351,-8.52651e-14
8,PK,0.945827,1.35287e-11
9,CONS,199.82,4.83169e-13
